In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import datetime
from sklearn.preprocessing import LabelEncoder
#한글깨짐방지
plt.rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

from IPython.core.display import display, HTML
display(HTML('<style>.container {width:100% !important; }</style>'))

In [91]:
path = './data/'
data_4 = pd.read_csv(path +'202004.csv')
# data = pd.read_csv(path +'201901-202003.csv')
submission = pd.read_csv(path + 'submission.csv')

In [34]:
data_4

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,3,779600,4
1,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,2,3,14,2427700,15
2,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,50s,1,4,18,2184600,18
3,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,50s,2,4,21,3653100,23
4,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,60s,1,5,15,2765500,18
...,...,...,...,...,...,...,...,...,...,...,...,...
1350317,202004,충북,충주시,휴양콘도 운영업,충북,충주시,20s,1,1,5,77000,5
1350318,202004,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,6,92000,6
1350319,202004,충북,충주시,휴양콘도 운영업,충북,충주시,40s,2,3,5,193000,5
1350320,202004,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,5,86000,7


### 제출파일생성
* 리더보드 확인용

In [22]:
grouped = data_4.groupby(['CARD_SIDO_NM','STD_CLSS_NM'])['AMT'].sum().reset_index()
grouped['REG_YYMM'] = 202004

In [26]:
submissioned = pd.merge(submission.drop('AMT',axis=1),grouped,how='left')
submissioned = submissioned.fillna(0)

In [33]:
submissioned.to_csv('only_4.csv', encoding='utf-8-sig',index=False)

In [30]:
submissioned[(submissioned.REG_YYMM == 202004)&(submissioned.AMT == 0)]

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.0
14,14,202004,강원,면세점,0.0
15,15,202004,강원,버스 운송업,0.0
30,30,202004,강원,정기 항공 운송업,0.0
45,45,202004,경기,그외 기타 분류안된 오락관련 서비스업,0.0
...,...,...,...,...,...
660,660,202004,충북,그외 기타 분류안된 오락관련 서비스업,0.0
670,670,202004,충북,면세점,0.0
671,671,202004,충북,버스 운송업,0.0
679,679,202004,충북,여행사업,0.0


### 평가지표확인

In [92]:
data_1904 = pd.read_csv('data_1904.csv')

In [96]:
data_1904['REG_YYMM'] = 202004

In [102]:
sub = submission[submission['REG_YYMM'] == 202004]

In [103]:
del sub['AMT']

In [109]:
data_1904_m = pd.merge(sub,data_1904,how='left')
data_1904_m = data_1904_m.fillna(0)

In [110]:
data_1904_m

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,1.065485e+08
1,1,202004,강원,골프장 운영업,3.964893e+09
2,2,202004,강원,과실 및 채소 소매업,8.276834e+08
3,3,202004,강원,관광 민예품 및 선물용품 소매업,1.790561e+07
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...,...
692,692,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.428269e+09
693,693,202004,충북,한식 음식점업,2.216287e+10
694,694,202004,충북,호텔업,4.051093e+07
695,695,202004,충북,화장품 및 방향제 소매업,6.872543e+08


In [97]:
def set_sumit(only_4, pre_df):
    '''리더보드와 동일한 예측을 하는 함수
    parameter
    --------
    only_4(df) : 4월 RMSLE가 0이 나왔던 제출셋
    pre_df(df) : 모델로 예측한 Dataframe
    
    '''
    from sklearn.metrics import mean_squared_log_error
    
    pre_df = pre_df[pre_df['REG_YYMM'] == 202004]
    only_4 = only_4[only_4['REG_YYMM'] == 202004]
    
    pre_df.loc[pre_df.CARD_SIDO_NM == '제주','weight'] = 3
    pre_df.loc[pre_df.CARD_SIDO_NM != '제주','weight'] = 1
    
    weight = pre_df['weight'].values
    
    all_amt = np.sqrt(mean_squared_log_error(only_4['AMT'],pre_df['AMT'],sample_weight=weight))
    return all_amt

In [111]:
set_sumit(only_4, data_1904_m)

3.3071904353033217

In [114]:
data_1904_m['AMT_19'] = data_1904_m['AMT']

In [115]:
del data_1904_m['AMT']

In [116]:
data_1904_m

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT_19
0,0,202004,강원,건강보조식품 소매업,1.065485e+08
1,1,202004,강원,골프장 운영업,3.964893e+09
2,2,202004,강원,과실 및 채소 소매업,8.276834e+08
3,3,202004,강원,관광 민예품 및 선물용품 소매업,1.790561e+07
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...,...
692,692,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.428269e+09
693,693,202004,충북,한식 음식점업,2.216287e+10
694,694,202004,충북,호텔업,4.051093e+07
695,695,202004,충북,화장품 및 방향제 소매업,6.872543e+08


In [119]:
pd.merge(only_4,data_1904_m).to_csv('19년vs20년_4월.csv',index = False,encoding = 'cp949')